In [1]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import plotly.express as px
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: f'%.{2}f' % x)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('../8Jul2020-14Jul2020.csv')
df['Date'] = pd.to_datetime(df['Start Date'], dayfirst=True)
df.sort_values(by='Date', inplace=True)
df['Date'] = df['Date'].dt.date
df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks,Date
0,99188805,780,13656,08/07/2020 00:13,476,"Stebondale Street, Cubitt Town",08/07/2020 00:00,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,2020-07-08
1,99188806,180,4393,08/07/2020 00:03,531,"Twig Folly Bridge, Mile End",08/07/2020 00:00,517,"Ford Road, Old Ford",51.53,-0.04,6,6,0,15,21,51.53,-0.03,2,2,0,16,18,2020-07-08
2,99188809,960,8973,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08
3,99188810,960,7141,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08
4,99188807,600,15085,08/07/2020 00:11,201,"Dorset Square, Marylebone",08/07/2020 00:01,797,"Ossulston Street, Somers Town",51.52,-0.16,15,15,0,1,16,51.53,-0.13,28,27,1,2,30,2020-07-08


In [3]:
# temp = df.groupby(['Date', 'EndStation Name'])['EndStation nbBikes'].agg(['first', 'last']).diff(axis=1)
# temp1 = df.groupby(['Date', 'EndStation Name'])['EndStation nbBikes'].agg(['first'])
temp1 = df.groupby(['EndStation Name'])['EndStation nbBikes'].agg(['first'])
# temp1.reset_index(drop=False, inplace=True)

# temp.drop(['first'], inplace=True, axis=1)
# temp.rename(columns = {'last':'Diff'}, inplace = True)
temp1.head()

,first
EndStation Name,
"Abbey Orchard Street, Westminster",15
"Abbotsbury Road, Holland Park",8
"Aberdeen Place, St. John's Wood",1
"Aberfeldy Street, Poplar",12
"Abingdon Green, Westminster",14


In [4]:
df = pd.read_csv('../8Jul2020-14Jul2020.csv')
df['Date'] = pd.to_datetime(df['Start Date'], dayfirst=True)
df.sort_values(by='Date', inplace=True)
df['Date'] = df['Date'].dt.date
df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks,Date
0,99188805,780,13656,08/07/2020 00:13,476,"Stebondale Street, Cubitt Town",08/07/2020 00:00,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,2020-07-08
1,99188806,180,4393,08/07/2020 00:03,531,"Twig Folly Bridge, Mile End",08/07/2020 00:00,517,"Ford Road, Old Ford",51.53,-0.04,6,6,0,15,21,51.53,-0.03,2,2,0,16,18,2020-07-08
2,99188809,960,8973,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08
3,99188810,960,7141,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08
4,99188807,600,15085,08/07/2020 00:11,201,"Dorset Square, Marylebone",08/07/2020 00:01,797,"Ossulston Street, Somers Town",51.52,-0.16,15,15,0,1,16,51.53,-0.13,28,27,1,2,30,2020-07-08


In [5]:
# temp = df.groupby(['Date', 'EndStation Name'])['EndStation nbBikes'].agg(['first', 'last']).diff(axis=1)
# temp2 = df.groupby(['Date', 'EndStation Name'])['EndStation nbBikes'].agg(['last'])
temp2 = df.groupby(['EndStation Name'])['EndStation nbBikes'].agg(['last'])
# temp2.reset_index(drop=False, inplace=True)

# temp.drop(['first'], inplace=True, axis=1)
# temp.rename(columns = {'last':'Diff'}, inplace = True)
temp2.head()

,last
EndStation Name,
"Abbey Orchard Street, Westminster",15
"Abbotsbury Road, Holland Park",8
"Aberdeen Place, St. John's Wood",1
"Aberfeldy Street, Poplar",12
"Abingdon Green, Westminster",14


In [6]:
temp = pd.concat([temp1,temp2], axis=1)
temp.dropna(inplace=True)
temp['Diff'] = temp['last'] - temp['first']
temp

,first,last,Diff
EndStation Name,,,
"Abbey Orchard Street, Westminster",15,15,0
"Abbotsbury Road, Holland Park",8,8,0
"Aberdeen Place, St. John's Wood",1,1,0
"Aberfeldy Street, Poplar",12,12,0
"Abingdon Green, Westminster",14,14,0
...,...,...,...
"Wren Street, Holborn",6,6,0
"Wright's Lane, Kensington",5,5,0
"Wynne Road, Stockwell",19,19,0
